Published on December 12, 2023. By Marília Prata, mpwolke 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/en-fr-translation-dataset/en-fr.csv
/kaggle/input/mental-health-chatbot-pairs/train.csv


#Citation: 

BigScience, BigScience Language Open-science Open-access Multilingual (BLOOM) Language Model. International, May 2021-May 2022

" BLOOM is able to generate text in 46 natural languages and 13 programming languages. For almost all of them, such as Spanish, French and Arabic, BLOOM will be the first language model with over 100B parameters ever created. This is the culmination of a year of work involving over 1000 researchers from 70+ countries and 250+ institutions, leading to a final run of 117 days training the BLOOM model on the Jean Zay supercomputer in the south of Paris, France thanks to a compute grant worth an estimated €3M from French research agencies CNRS and GENCI."

https://bigscience.huggingface.co/blog/bloom

#We have Human and Assistant

In [2]:
df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')
df.tail()

,en,fr
22520371,Only with a highly overcompensatory stock–recr...,C'est seulement en cas de courbe stock–recrute...
22520372,The model predicts that the assumption made ab...,Le modèle prévoit que l'hypothèse émise au suj...
22520373,Overall the results confirm the unsatisfactory...,"Dans l'ensemble, les résultats confirment le p..."
22520374,"Error 404 — file not found Sorry, but the file...",Erreur 404 — fichier introuvable Nous sommes d...
22520375,British Columbia Lodging and Campgrounds Assoc...,British Columbia Lodging and Campgrounds Assoc...


BLOOM: A 176B-Parameter Open-Access Multilingual Language Model

Authors: Probably, that's the longest Authors list I've ever seen. There are 3 pages with their names on the pdf document.

BigScience Workshop: Teven Le Scao, Angela Fan, Christopher Akiki, Ellie Pavlick, Suzana Ilić, Daniel Hesslow, Roman Castagné, Alexandra Sasha Luccioni, François Yvon, Matthias Gallé, Jonathan Tow, Alexander M. Rush, Stella Biderman, Albert Webson, Pawan Sasanka Ammanamanchi, Thomas Wang, Benoît Sagot, Niklas Muennighoff, Albert Villanova del Moral, Olatunji Ruwase, Rachel Bawden, Stas Bekman, Angelina McMillan-Major, Iz Beltagy, Huu Nguyen, Lucile Saulnier, Samson Tan, Pedro Ortiz Suarez, Victor Sanh, Hugo Laurençon, Yacine Jernite, Julien Launay, Margaret Mitchell, Colin Raffel, Aaron Gokaslan, Adi Simhi, Aitor Soroa, Alham Fikri Aji, Amit Alfassy, Anna Rogers, Ariel Kreisberg Nitzav, Canwen Xu, Chenghao Mou, Chris Emezue, Christopher Klamm, Colin Leong, Daniel van Strien, David Ifeoluwa Adelani, Dragomir Radev, Eduardo González Ponferrada, Efrat Levkovizh, Ethan Kim, Eyal Bar Natan, Francesco De Toni, Gérard Dupont, Germán Kruszewski, Giada Pistilli, Hady Elsahar, Hamza Benyamina, Hieu Tran, Ian Yu, Idris Abdulmumin, Isaac Johnson, Itziar Gonzalez-Dios, Javier de la Rosa, Jenny Chim, Jesse Dodge, Jian Zhu, Jonathan Chang, Jörg Frohberg, Joseph Tobing, Joydeep Bhattacharjee, Khalid Almubarak, Kimbo Chen, Kyle Lo, Leandro Von Werra, Leon Weber, Long Phan, Loubna Ben allal, Ludovic Tanguy, Manan Dey, Manuel Romero Muñoz, Maraim Masoud, María Grandury, Mario Šaško, Max Huang, Maximin Coavoux, Mayank Singh, Mike Tian-Jian Jiang, Minh Chien Vu, Mohammad A. Jauhar, Mustafa Ghaleb, Nishant Subramani, Nora Kassner, Nurulaqilla Khamis, Olivier Nguyen, Omar Espejel, Ona de Gibert, Paulo Villegas et al. (293 additional authors not shown)

"Pretrained language models have become a cornerstone of modern natural language processing (NLP) pipelines because they often produce better performance from smaller quantities of labeled data. The development of ELMo, ULMFiT (Howard and Ruder, 2018), GPT (Radford et al., 2018), and BERT led to the widespread use of pretrained models as an initialization for finetuning on downstream tasks."

"The subsequent finding that pretrained language models can perform useful tasks without
any additional training) further demonstrated their utility. In addition, the empirical observation that a language model’s performance tends to increase as the model is made larger—sometimes predictably and sometimes suddenly has led to a  trend of increasing scale. Apart from environmental concerns, the costs of training large language models (LLMs) are only affordable for well-resourced organizations."

"As a result, the majority of the research community has been excluded from the development of LLMs. This exclusion has had concrete consequences; for example, most LLMs are primarily trained on English-language text (with notable exceptions in Chinese and Korean).

"BLOOM is a 176 billion parameter language model trained on 46 natural languages and 13 programming languages that was developed and released by a collaboration of hundreds of researchers. The compute for training BLOOM was provided through a French public grant from GENCI and IDRIS, leveraging IDRIS’ Jean Zay supercomputer. To build BLOOM, the authors undertook a thorough design process for each of its components, including the training dataset, model architecture and training objective, and engineering strategy for distributed learning. The authors also performed an analysis of the model’s capabilities. Their overall aim is not only to publicly release a large-scale multilingual language model with performance comparable to recently developed systems, but also to document the coordinated process that went into its development. The purpose of this paper is to provide a high-level overview of these design steps while referencing the individual reports they produced over the course of developing BLOOM."

https://arxiv.org/abs/2211.05100

In [8]:
#4th row, 1st column   

df.iloc[44,0]

'Instruments The ancient Egyptians use the transit for the first time about 4,000 years ago.'

#Social Limitations of LLM Development

"While the continued increase in the size of large language models has resulted in improvements across a wide range of tasks, it has also exacerbated issues with their development and use. The computational expense of large models also prohibits the majority of the research community from participating in their development, evaluation and routine use. Moreover, the computational costs have also lead to concerns about the carbon footprint stemming from the training and use of large language models, and existing carbon footprint studies have likely under-estimated emissions. Contributing to an increase in the global carbon footprint exacerbates climate change which most severely affects already-marginalized communities. Furthermore, the concentration of resources within a handful of (typically industrial) institutions with primarily technical expertise hinders prospects for an inclusive, collaborative, and reliable governance of the technology."

"Despite the substantial social dangers in allowing this technology to be developed unilaterally by corporations, EleutherAI was the only non-corporate entity outside of China that was developing large language models before the BigScience Workshop was convened."

https://arxiv.org/pdf/2211.05100.pdf

In [9]:
import torch

#Ethical Considerations within BigScience 

"In order to acknowledge and start addressing social limitations of LLM development within BigScience, the workshop relied on a collaboratively designed Ethical Charter and original research on applicable regulations in jurisdictions outside of the US to guide its choices throughout the project. In particular, the charter emphasizes values of inclusivity and diversity, openness and reproducibility, and responsibility in various aspects of the organization. Each of these values are showcased in different ways in the dataset curation, modeling, engineering, evaluation, and other social impact (throughout) aspects of the project."

"BLOOM was trained on the ROOTS corpus, a composite collection of 498 Hugging Face datasets amounting to 1.61 terabytes of text that span 46 natural languages and 13 programming languages."

MOTIVATION - Under-valued "Data Work"-“High-quality” data for as little cost 

"The disconnect between developers and (in)voluntary users of the technology is particularly apparent in the curation of the datasets that have supported recent large-scale machine learning projects, where intentional “Data work” is generally under-valued."

"In the context of LLMs, this tendency is exemplified by a range of heuristics-based filtering approaches that prioritize getting as much “high-quality” data for as little cost as possible over engaging with the needs—and rights—of data subjects, where quality is commonly defined as maximizing performance on downstream tasks while occasionally removing content deemed offensive by the developers."

LANGUAGE CHOICES

"These considerations led us to an incremental process for choosing which languages were to be included in the corpus. The authors started with a list of eight of the world’s largest languages by number of speakers for which they did active outreach in the
early stages of the project to invite fluent speakers to join the data efforts. Then, on the recommendation of language communities they expanded Swahili in the original selection to the category of Niger-Congo languages, and Hindi and Urdu to Indic languages.

"Finally, the authors proposed that any group of 3 or more participants fluent in an additional language could add it to the supported list if they would commit to selecting sources and guiding processing choices in the language in order to avoid common issues with corpora selected through automatic language identification without specific language expertise."

"QUALITY" FILTERING: TEXT PRODUCED by Humans For HUMANS 

"After obtaining the text, we found that most of the sources contained some amount of text that was not naturallanguage, for example preprocessing errors, SEO pages, or spam (including pornographic spam)."

" In order to filter non-natural language, the authors defined a set of quality indicators, where high-quality text is defined as “written by humans for humans”, without distinction of content (as they wanted content selection to exclusively be the domain of the more accountable human source selection) or a priori judgments of grammaticality."

https://arxiv.org/pdf/2211.05100.pdf

In [10]:
# Set up the GPU device
device = torch.device("cuda")

#Downloading the model bigscience/bloom-1b7

In [11]:
#By Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

from transformers import BloomTokenizerFast, BloomForCausalLM

tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-1b7")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-1b7")

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

#PROMPTS
 
"Based on recent research on the impact of prompting on language model performance, the authors decided to build a language model evaluation suite that allowed them to vary both the basic task data as well as the prompting that is used to contextualize the task. Their prompts were developed prior to BLOOM’s release, and did not undergo any a priori refinement using models. That is, the prompts they used in their evaluation are ones that humans believed were a reasonable way to solicit the desired task behavior from a language model. Their goal for designing prompts in this way is to simulate realistic zero-shot or one-shot results that a new user could expect from BLOOM. This is in contrast to presenting best-case performances that might result from multiple rounds of trial-and-error on prompt design."

"The authors chose to report the former because the latter is harder to reproduce systematically, is arguably a less representative picture of how the model works in the average setting, and is not representative of true zero-shot learning where no labeled data is available. They generated multiple prompts per task using promptsource."

"They followed the procedure, in which prompt generation is crowd-sourced, and thus they saw substantial variety in length and style across prompts. To improve quality and clarity, multiple peer reviews were performed on each prompt for artifacts and consistency."

"They also generated prompts for many tasks that are not included in this paper due to resource constraints. All of their prompts for all tasks (both those analyzed in the paper and those not yet analyzed) are publicly available."

On Baseline models:  T0, a variant of T5 that underwent multitask prompted finetuning on datasets from P3

#WMT - Best Prompts, the more Verbose ones

"The best prompts tend to be the more verbose ones; the “version-target” prompt is consistently better and the “gpt3-target” and “xglm-source+target” prompts have very poor performance, especially for zero-shot."

"In the one-shot setting, BLOOM can, with the right prompt, perform competent translation, although it is behind dedicated (supervised) models such as M2M-100 (43.8 BLEU for English→French and 40.4 for French→English, compared to 34.2 and 35.4 BLEU for BLOOM). The two major problems observed, particularly in the zero-shot setting, are (i) over-generation and (ii) not producing the correct language (an obvious prerequisite for a good translation). Both of these aspects are greatly improved as the number of few-shot examples is increased."

https://arxiv.org/pdf/2211.05100.pdf


#Turn ON GPU!

In [12]:
model = model.to(device)

#Multilingual Probing

"Probing has emerged as a significant evaluation paradigm to analyze and interpret the inner workings of LLMs although it comes with certain shortcomings. Examination of the LLM embeddings can help shed light on the generalizing abilities of the model apart from its training objective loss or downstream task evaluation, which is especially beneficial for examining languages lacking annotated datasets or benchmarks."

Correlation

"The authors run statistical tests to analyze correlations between the probing performance and linguistic, dataset, and model configuration criteria:"

Probing and pretraining dataset size: the authors run the Pearson correlation coefficient test (Pearson, 1895) to compute correlation between the probing performance and these
data configuration criteria."

"Effect of the model size: the results are divided into two groups by the BLOOM
version. Here, they used the Mann-Whitney U test to see if there is a correlation between
the number of parameters and the probing results."

#Probing Results

"The results of probing experiments averaged over the probing tasks and experiment runs within each language. The overall pattern is that BLOOM-1B7 performs on par or better than BLOOM, and both LLMs outperform the count-based baselines. In particular, the LLMs achieve more robust performance on Arabic, Basque, and Indo-European languages (e.g., Catalan, French, Hindi, Portuguese, Spanish, and Urdu), while Bengali, Wolof, and Yoruba receive the lowest scores. The authors attributed this behavior to the transfer abilities: BLOOM infers linguistic properties better for the closely related languages that comprise a significant amount of data." 

Multilingual Abilities.

"A separate research interest implies considering languages that are not explicitly included in the pretraining corpus of the models. Expanding the set of languages for probing will allow for a typological interpretation and a deeper analysis of the most learnable and hard-to-learn linguistic features on a more considerable scope."

https://arxiv.org/pdf/2211.05100.pdf

In [20]:
text = df.loc[44,'fr']

In [24]:
en_text = df.loc[44,'en']

In [21]:
text

'Instruments Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ.'

In [25]:
en_text

'Instruments The ancient Egyptians use the transit for the first time about 4,000 years ago.'

In [22]:
#By Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

inputs = tokenizer.encode(text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)

In [23]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Instruments Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000 ans avant Jésus-Christ. Les Égyptiens utilisent le transit pour la première fois, environ 2000


In [26]:
#By Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

inputs = tokenizer.encode(en_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Instruments The ancient Egyptians use the transit for the first time about 4,000 years ago. The Egyptians were the first people to use the transit for the first time about 4,000 years ago. The Egyptians were the first people to use the transit for the first time about 4,000 years ago. The Egyptians were the first people to use the transit for the first time about 4,000 years ago. The Egyptians were the first people to use the transit for the first time about 4,000 years ago. The Egyptians were the first people to use the transit for the first time about 4,000 years ago. The Egyptians were the first people to use the transit for the first time about 4,


#The difference is on the Last paragraph:

It has one more line: "If you or a loved one is struggling with a mental health condition, reach out to us today for a free consultation."

In [ ]:
#By Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

print(tokenizer.decode(outputs[2], skip_special_tokens=True))#Original was outputs[1]

In [ ]:
text2 = df.loc[21, "text"]

In [ ]:
text2

In [ ]:
#By Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

inputs = tokenizer.encode(text2, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#Difference from above. That was added after "the client and with his consent."

#Check input 9. Compare with those lines. Where did the bigscience/bloom-1b7 take that???

" The therapist and the client work together to develop a treatment plan that is tailored to the needs of the client and his family.
<HUMAN>: What is the role of the therapist in the treatment of addictions?
<ASSISTANT>: The role of the therapist in the treatment of addictions is twofold. First, the therapist assists the client and his family in developing a treatment plan that is tailored to the needs of the client and his family. Second, the therapist works closely with the client and his family to ensure that the client and his family receive the support they need to overcome the challenges."

In [ ]:
#By Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

print(tokenizer.decode(outputs[3], skip_special_tokens=True))#Original was outputs[1]

#Acknowledgements:

Man of the year https://www.kaggle.com/code/manwithaflower/bloom-1b7-gsm8k

mpwolke  https://www.kaggle.com/code/mpwolke/socraticgsm8k-with-bloom-1b7